# Fuera de esta dimensión

En este trabajo, entraré a la competencia de Kaggle llamada *Spaceship Titanic* (https://www.kaggle.com/competitions/spaceship-titanic) para predecir qué individuos han sido expulsados a otra dimensión tras un accidente espacial utilizando un modelo de clasificación. Para ello, entrenaré modelos de regresión logística, LDA, DTs, RFs, y Adaboost, y tras escoger uno, realizaré predicciones y las enviaré al sitio de Kaggle para que evaluen la precisión de las predicciones. 

A lo largo de este trabajo se encontrarán referencias in-code acerca de uso de IA en el mismo.

Veamos las variables:

- ``PassengerId``: *A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always*.
    - no se si es categorica o cuantitiva. es único tho
- ``HomePlanet``: *The planet the passenger departed from, typically their planet of permanent residence.*
    - categorica: categoria
- ``CryoSleep``: *Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.*
    - categorica: booleana
- ``Cabin``: *The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.*
    - categorica: categoria
- ``Destination``: *The planet the passenger will be debarking to.*
    - categorica: categoria
- ``Age``: *The age of the passenger.*
    - cuantitiva
- ``VIP``: *Whether the passenger has paid for special VIP service during the voyage.*
    - categorica: booleana
- ``RoomService, FoodCourt, ShoppingMall, Spa, VRDeck``: *Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.*
    - cuanatitva
- ``Name``: *The first and last names of the passenger.*
    - categorica: categoria
- ``Transported``: *Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.*
    - categorica: booleana
    - TARGET

## Exploracion de datos

Quiero analizar:
- tamaño de datos
- tipo de datos
- huecos

Examinemos primero el tamaño de los datos. Espero ~8700 observaciones

In [303]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")

print(f"Tamaño dfTrain: {dfTrain.shape}")

Tamaño dfTrain: (8693, 14)


 Ahora su tipo de datos:

### huecos

Veamos si hay valores NaN en alguna columna:

Okay son muchos y no quiero arriesgarme a borrar 2000 observaciones (asumiendo que son cada una distintas como worst case) ni tampoco rellenar por rellenar, analizaré las variables de la base de datos y usaré una metodología para rellenar huecos

In [304]:
print(f"Valores NaN por columna en datos x: {dfTrain.isna().sum()}")

Valores NaN por columna en datos x: PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


In [305]:
"""
ESTAS INSTRUCCIONES LE DI A GITHUB COPILOT AI:

ocupo que programes en camelCase y en español

pon esta linea de codigo: nombresUnicos = dfTrain.Name.value_counts()[dfTrain.Name.value_counts() == 1].index 

Luego, necesito rellenar huecos en una base de datos 'dfTrain'. Hay 14 variables. Para las siguientes variables quiero que tomes las siguients acciones:

1. [RoomService, FoodCourt, ShoppingMail, Spa, VRDeck]: Si hay hueco en alguna de estas 5, quiero que tomes la moda de las otras 4 de la misma observación. Si no hay moda, toma el promedio de las otras 4.
2. [HomePlanet, Destination, Cabin, VIP]: toma la moda de las 3 observaciones anteriores y 3 posteriores a la actual.
3. Name: 
    si hay una moda (es decir, si algun valor se repite) en los 3 cabins antes y despues de ese:
        - obtener primer nombre de uno de los únicos
        - obtener segundo nombre de alguno de los de la media
        - agregar nombre completo en observacion actual, conformado por el primer nombre y segundo nombre obtenidos
    si no:
        agarra nombre completo uno de los que aparecen ndms una vez al azar y ponlo también en la observación actual
4. Cryosleep: 
    si hay puros ceros en alguna de las variables [RoomService, FoodCourt, ShoppingMail, Spa, VRDeck]:
        pon este valor como True.
    Si no:
        ponle que False 
5. Age: pon el promedio de las tres observaciones anteriores y los tres siguientes

"""


    
# Obtener nombres únicos
nombresUnicos = dfTrain.Name.value_counts()[dfTrain.Name.value_counts() == 1].index

# 1. Rellenar [RoomService, FoodCourt, ShoppingMail, Spa, VRDeck]
servicios = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for index, row in dfTrain.iterrows():
    for servicio in servicios:
        if pd.isna(row[servicio]):
            otrosServicios = [row[s] for s in servicios if s != servicio and not pd.isna(row[s])]
            if len(otrosServicios) > 0:
                moda = pd.Series(otrosServicios).mode()
                if len(moda) > 0:
                    dfTrain.at[index, servicio] = moda[0]
                else:
                    dfTrain.at[index, servicio] = np.mean(otrosServicios)

# 2. Rellenar [HomePlanet, Destination, Cabin, VIP]
variablesCategoricas = ['HomePlanet', 'Destination', 'Cabin', 'VIP']
for index in range(len(dfTrain)):
    for variable in variablesCategoricas:
        if pd.isna(dfTrain.at[index, variable]):
            inicio = max(0, index - 3)
            fin = min(len(dfTrain), index + 4)
            moda = dfTrain[variable][inicio:fin].mode()
            if len(moda) > 0:
                dfTrain.at[index, variable] = moda[0]             
                
# 3. Rellenar Name
for index in range(len(dfTrain)):
    if pd.isna(dfTrain.at[index, 'Name']):
        inicio = max(0, index - 3)
        fin = min(len(dfTrain), index + 4)
        modaCabin = dfTrain['Cabin'][inicio:fin].mode()
        if len(modaCabin) > 0:
            primerNombre = np.random.choice(nombresUnicos).split()[0]
            segundoNombre = np.random.choice(dfTrain['Name'][inicio:fin].dropna().values).split()[1]
            dfTrain.at[index, 'Name'] = f"{primerNombre} {segundoNombre}"
        else:
            dfTrain.at[index, 'Name'] = np.random.choice(nombresUnicos)  

# 4. Rellenar Cryosleep
for index, row in dfTrain.iterrows():
    if pd.isna(row['CryoSleep']):
        if all(row[servicio] == 0 for servicio in servicios):
            dfTrain.at[index, 'CryoSleep'] = True
        else:
            dfTrain.at[index, 'CryoSleep'] = False     

# 5. Rellenar Age
for index in range(len(dfTrain)):
    if pd.isna(dfTrain.at[index, 'Age']):
        inicio = max(0, index - 3)
        fin = min(len(dfTrain), index + 4)
        dfTrain.at[index, 'Age'] = dfTrain['Age'][inicio:fin].mean()

print(f"\nValores NaN por columna: {dfTrain.isna().sum()}")
print(f"\nTamaño dfTrain: {dfTrain.shape}")
print(f"\nTipo de datos dfTrain: {dfTrain.dtypes}")


Valores NaN por columna: PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

Tamaño dfTrain: (8693, 14)

Tipo de datos dfTrain: PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object


Como se puede ver, **se eliminaron todos los huecos y se mantuvo igual el tamaño de los datos**, por lo que vemos que no hubo una eliminación indeseada de datos. Claro que la discusión de si la metodología para hacerlo es razonable, o en todo caso, la mejor, es un tema aparte.

### Variables categóricas

Tras haber eliminado los huecosHay variables con demasiada cardinalidad: ``PassengerId``, ``Cabin`` y ``Name``. Las eliminaré del df

In [306]:
dfTrain = dfTrain.drop(["PassengerId", "Cabin", "Name"], axis=1)

habran modelos que trabajen mejor tras haber sido tratados con OHE, y otros que no. Por eso crearé dos dataFrames distintos. Uno para cada grupo.

In [307]:
"""GITHUB COPILOT"""

oheDFTrain = dfTrain
nombresDatosCategoricos = ["HomePlanet", "CryoSleep", "Destination", "VIP", "Transported"]
indicesDatosCategoricos = oheDFTrain.columns.get_indexer(nombresDatosCategoricos)

# trabajado con IA
for i in indicesDatosCategoricos:
    original_column_name = oheDFTrain.columns[i]
    dummy = pd.get_dummies(oheDFTrain.iloc[:,i], prefix=original_column_name, drop_first=True, dtype=np.int64)
    oheDFTrain = pd.concat([oheDFTrain,dummy],axis=1)

# normal
oheDFTrain = oheDFTrain.drop(nombresDatosCategoricos, axis=1)

print(oheDFTrain.head())

    Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0  39.0          0.0        0.0           0.0     0.0     0.0   
1  24.0        109.0        9.0          25.0   549.0    44.0   
2  58.0         43.0     3576.0           0.0  6715.0    49.0   
3  33.0          0.0     1283.0         371.0  3329.0   193.0   
4  16.0        303.0       70.0         151.0   565.0     2.0   

   HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  \
0                  1                0               0   
1                  0                0               0   
2                  1                0               0   
3                  1                0               0   
4                  0                0               0   

   Destination_PSO J318.5-22  Destination_TRAPPIST-1e  VIP_True  \
0                          0                        1         0   
1                          0                        1         0   
2                          0                        1         1  

In [308]:
print(oheDFTrain.dtypes)

Age                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
HomePlanet_Europa              int64
HomePlanet_Mars                int64
CryoSleep_True                 int64
Destination_PSO J318.5-22      int64
Destination_TRAPPIST-1e        int64
VIP_True                       int64
Transported_True               int64
dtype: object


## Regresión logística

Escalemos los datos para que el modelo pueda funcionar adecuadamente

In [309]:
from sklearn.preprocessing import StandardScaler

x = oheDFTrain[oheDFTrain.columns[oheDFTrain.columns != "Transported_True"]]
y = oheDFTrain["Transported_True"]

# AI
scaler = StandardScaler()
xEscalado = scaler.fit_transform(x)
xEscalado = pd.DataFrame(xEscalado, columns=x.columns)

Haré una regresión logística *multiple*, para predecir 0 o 1 a partir de las variables de entrada

In [310]:
# Tras leer docus de KFold y ayuda de IA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score


xTrain, xTest, yTrain, yTest = train_test_split(xEscalado, y, test_size=0.3, random_state=27, stratify=y) 
rlModelo = LogisticRegression()
rlModelo.fit(xTrain, yTrain)

kf = KFold(n_splits=5, shuffle=True)
rlCVScores = cross_val_score(rlModelo, xTrain, yTrain, cv=kf)
print(rlCVScores)

[0.76253081 0.79211175 0.79457683 0.79293344 0.79046836]


In [311]:
"""AI """
# Calcular el promedio y la desviación estándar
promedioCV = rlCVScores.mean()
desviacionCV = rlCVScores.std()

print(f"Promedio de validación cruzada: {promedioCV:.4f}")
print(f"Desviación estándar de validación cruzada: {desviacionCV:.4f}")

# Evaluar el modelo en el conjunto de prueba
accuracyTest = rlModelo.score(xTest, yTest)
print(f"Exactitud en el conjunto de prueba: {accuracyTest:.4f}")

from sklearn.metrics import classification_report, confusion_matrix

# Predicciones en el conjunto de prueba
yPred = rlModelo.predict(xTest)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(yTest, yPred))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(yTest, yPred))

Promedio de validación cruzada: 0.7865
Desviación estándar de validación cruzada: 0.0121
Exactitud en el conjunto de prueba: 0.7791
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.78      0.77      0.78      1295
           1       0.78      0.79      0.78      1313

    accuracy                           0.78      2608
   macro avg       0.78      0.78      0.78      2608
weighted avg       0.78      0.78      0.78      2608

Matriz de confusión:
[[1001  294]
 [ 282 1031]]


## LDA

In [312]:
from statsmodels.api import GLM
from statsmodels.genmod.families import Binomial

resultados = GLM(yTrain, xTrain, family=Binomial()).fit()
print(resultados.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:       Transported_True   No. Observations:                 6085
Model:                            GLM   Df Residuals:                     6073
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2728.2
Date:                Mon, 31 Mar 2025   Deviance:                       5456.4
Time:                        15:21:28   Pearson chi2:                 5.79e+03
No. Iterations:                     7   Pseudo R-squ. (CS):             0.3871
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Age                   

In [313]:
### HECHO CON GITHUB COPILOT -- Test para evaluar relevancia del modelo
from scipy.stats import chi2

# Get the null deviance and model deviance
null_deviance = resultados.null_deviance
model_deviance = resultados.deviance

# Degrees of freedom
df_diff = resultados.df_model

# Calculate the p-value
p_value = chi2.sf(null_deviance - model_deviance, df_diff)
print(f"Valor P para significancia general del modelo: {p_value}")

Valor P para significancia general del modelo: 0.0


Todas las variables tienen aportaciones significativas a la variable de salida. Por tanto, habrá que hacer un LDA con todas.

In [314]:
### HECHO CON COPILOT
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns

# Entrena el modelo de LDA
lda = LinearDiscriminantAnalysis()
lda.fit(xTrain, yTrain)

ldaCVScores = cross_val_score(lda, xTrain, yTrain, cv=kf)
print(ldaCVScores)

[0.76581758 0.76992605 0.76581758 0.76170912 0.76910435]


In [315]:
"""AI """
# Calcular el promedio y la desviación estándar
promedioCV = ldaCVScores.mean()
desviacionCV = ldaCVScores.std()

print(f"Promedio de validación cruzada: {promedioCV:.4f}")
print(f"Desviación estándar de validación cruzada: {desviacionCV:.4f}")

# Evaluar el modelo en el conjunto de prueba
accuracyTest = lda.score(xTest, yTest)
print(f"Exactitud en el conjunto de prueba: {accuracyTest:.4f}")

# Predicciones en el conjunto de prueba
yPred = lda.predict(xTest)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(yTest, yPred))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(yTest, yPred))

Promedio de validación cruzada: 0.7665
Desviación estándar de validación cruzada: 0.0029
Exactitud en el conjunto de prueba: 0.7565
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.73      0.81      0.77      1295
           1       0.79      0.71      0.75      1313

    accuracy                           0.76      2608
   macro avg       0.76      0.76      0.76      2608
weighted avg       0.76      0.76      0.76      2608

Matriz de confusión:
[[1045  250]
 [ 385  928]]


### LDA vs Regresión logística

Es visible como el promedio de puntiación en CV es un poco mejor en la regresión logística que en LDA, aunque la regresión tiene mayor sensibilidad clase 1 que para clase 0, lo contrario a LDA.

además, en prueba de accuracy es un poco mejor en la regresión que en LDA

## Árbol de decisión

Ahora entrenemos un arbol de decision

In [316]:
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import plot_tree
from sklearn.model_selection import StratifiedKFold

tree = DTC().fit(xTrain, yTrain)

# se tienen muchas observaciones, entonces con pocos folds se puede generalizar bien para datos fuera de entrenamiento
skf = StratifiedKFold(n_splits=5)

### HECHO CON COPILOT
# Obtén el camino de poda basado en complejidad
path = tree.cost_complexity_pruning_path(xTrain, yTrain)

# Extrae los valores de ccp_alpha
ccp_alphas = path.ccp_alphas  # Valores de alpha relevantes
impurities = path.impurities  # Impurezas asociadas

cv_scores = []

ccp = np.linspace(0.001, ccp_alphas.max(), 150) 
for alpha in ccp:
    prunedTree = DTC(ccp_alpha=alpha)
    cv_scores.append(np.mean(cross_val_score(prunedTree, xTrain, yTrain, cv=skf, scoring='f1')))
alpha = ccp[np.argmax(cv_scores)]
print(f"Alpha optimo: {alpha}")

Alpha optimo: 0.001762825843151294


In [317]:
prunedTree = DTC(ccp_alpha=alpha).fit(xTrain, yTrain)
dtcCVScores = cross_val_score(prunedTree, xTrain, yTrain, cv=kf)
print(dtcCVScores)

[0.80115037 0.7748562  0.75924404 0.79046836 0.80115037]


In [318]:
"""AI """
# Calcular el promedio y la desviación estándar
promedioCV = dtcCVScores.mean()
desviacionCV = dtcCVScores.std()

print(f"Promedio de validación cruzada: {promedioCV:.4f}")
print(f"Desviación estándar de validación cruzada: {desviacionCV:.4f}")

# Evaluar el modelo en el conjunto de prueba
accuracyTest = prunedTree.score(xTest, yTest)
print(f"Exactitud en el conjunto de prueba: {accuracyTest:.4f}")

# Predicciones en el conjunto de prueba
yPred = prunedTree.predict(xTest)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(yTest, yPred))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(yTest, yPred))

Promedio de validación cruzada: 0.7854
Desviación estándar de validación cruzada: 0.0162
Exactitud en el conjunto de prueba: 0.7742
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.79      0.75      0.77      1295
           1       0.76      0.80      0.78      1313

    accuracy                           0.77      2608
   macro avg       0.77      0.77      0.77      2608
weighted avg       0.77      0.77      0.77      2608

Matriz de confusión:
[[ 967  328]
 [ 261 1052]]


Son resultados parecidos a ambas metodologías anteriores. En exactitud es un poco mejor que LDA, pero un poco peor que la regresión logística

## Random Forest

Ahora trabajremos con RF

In [319]:
"""Referencias de: https://www.youtube.com/watch?v=_QuGM_FW9eo"""

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(xTrain, yTrain)

rfCVScores = cross_val_score(rf, xTrain, yTrain, cv=kf)
print(rfCVScores)

[0.7863599  0.78471652 0.76335251 0.77403451 0.79046836]


In [320]:
"""AI """
# Calcular el promedio y la desviación estándar
promedioCV = rfCVScores.mean()
desviacionCV = rfCVScores.std()

print(f"Promedio de validación cruzada: {promedioCV:.4f}")
print(f"Desviación estándar de validación cruzada: {desviacionCV:.4f}")

# Evaluar el modelo en el conjunto de prueba
accuracyTest = rf.score(xTest, yTest)
print(f"Exactitud en el conjunto de prueba: {accuracyTest:.4f}")

# Predicciones en el conjunto de prueba
yPred = rf.predict(xTest)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(yTest, yPred))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(yTest, yPred))

Promedio de validación cruzada: 0.7798
Desviación estándar de validación cruzada: 0.0098
Exactitud en el conjunto de prueba: 0.7933
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.79      0.80      0.79      1295
           1       0.80      0.79      0.79      1313

    accuracy                           0.79      2608
   macro avg       0.79      0.79      0.79      2608
weighted avg       0.79      0.79      0.79      2608

Matriz de confusión:
[[1035  260]
 [ 279 1034]]


Esto es interesante, porque se obtuvo un resultado un poco mejor en exactitud en RF que con regresion logistica. También lo es en promedio de CV, pero al considerar sus promedios y sus desviaciones estándar son similares.

## AdaBoost

Experimentare con otro modelo. AdaBoost. 

In [321]:
"""referencia: https://www.youtube.com/watch?v=vlkhsbOATBo

es de xgboost pero de ahi agarre la idea de solo hacer el fit y el cv

"""
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier()
ab.fit(xTrain, yTrain)

abCVScores = cross_val_score(rf, xTrain, yTrain, cv=kf)
print(abCVScores)

[0.78471652 0.7863599  0.77814297 0.79622021 0.78307313]


In [322]:
"""AI """
# Calcular el promedio y la desviación estándar
promedioCV = abCVScores.mean()
desviacionCV = abCVScores.std()

print(f"Promedio de validación cruzada: {promedioCV:.4f}")
print(f"Desviación estándar de validación cruzada: {desviacionCV:.4f}")

# Evaluar el modelo en el conjunto de prueba
accuracyTest = ab.score(xTest, yTest)
print(f"Exactitud en el conjunto de prueba: {accuracyTest:.4f}")

from sklearn.metrics import classification_report, confusion_matrix

# Predicciones en el conjunto de prueba
yPred = ab.predict(xTest)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(yTest, yPred))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(yTest, yPred))

Promedio de validación cruzada: 0.7857
Desviación estándar de validación cruzada: 0.0059
Exactitud en el conjunto de prueba: 0.7500
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.71      0.83      0.77      1295
           1       0.80      0.67      0.73      1313

    accuracy                           0.75      2608
   macro avg       0.76      0.75      0.75      2608
weighted avg       0.76      0.75      0.75      2608

Matriz de confusión:
[[1075  220]
 [ 432  881]]


okay este modelo tuvo la peor performance en exactitud, pero probablemente hay otras cosas que se le pueden hacer para mejorarlo y no lo consideré.

## PREDICCIONES FINALES

### escala de datos

dado que transformé los datos para entrenar, necesito transformar los datos de entrada con los que voy a predecir, pero NO rellenaré huecos, sino tendría fuga de datos, creo.

In [324]:
dfTest = dfTest.drop(["PassengerId", "Cabin", "Name"], axis=1)

In [325]:
"""GITHUB COPILOT"""

oheDFTest = dfTest
nombresDatosCategoricos = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
indicesDatosCategoricos = oheDFTest.columns.get_indexer(nombresDatosCategoricos)

# trabajado con IA
for i in indicesDatosCategoricos:
    original_column_name = oheDFTest.columns[i]
    dummy = pd.get_dummies(oheDFTest.iloc[:,i], prefix=original_column_name, drop_first=True, dtype=np.int64)
    oheDFTest = pd.concat([oheDFTest,dummy],axis=1)

# normal
oheDFTest = oheDFTest.drop(nombresDatosCategoricos, axis=1)

print(oheDFTest.head())

    Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0  27.0          0.0        0.0           0.0     0.0     0.0   
1  19.0          0.0        9.0           0.0  2823.0     0.0   
2  31.0          0.0        0.0           0.0     0.0     0.0   
3  38.0          0.0     6652.0           0.0   181.0   585.0   
4  20.0         10.0        0.0         635.0     0.0     0.0   

   HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  \
0                  0                0               1   
1                  0                0               0   
2                  1                0               1   
3                  1                0               0   
4                  0                0               0   

   Destination_PSO J318.5-22  Destination_TRAPPIST-1e  VIP_True  
0                          0                        1         0  
1                          0                        1         0  
2                          0                        0         0  
3 

In [326]:
xTestReal = oheDFTest

In [327]:
"""AI """
# Escalar el conjunto de prueba real utilizando el scaler ajustado
xTestRealEscalado = scaler.transform(xTestReal)

# Convertir a DataFrame para mantener los nombres de las columnas (opcional)
xTestRealEscalado = pd.DataFrame(xTestRealEscalado, columns=xTestReal.columns)

# Ver los datos escalados
print(xTestRealEscalado.head())

        Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  \
0 -0.127954    -0.333158  -0.281070     -0.283630 -0.270688 -0.263009   
1 -0.684676    -0.333158  -0.275430     -0.283630  2.237567 -0.263009   
2  0.150407    -0.333158  -0.281070     -0.283630 -0.270688 -0.263009   
3  0.637539    -0.333158   3.887668     -0.283630 -0.109868  0.252837   
4 -0.615086    -0.318017  -0.281070      0.778304 -0.270688 -0.263009   

   HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  \
0          -0.575358        -0.508685        1.331498   
1          -0.575358        -0.508685       -0.751034   
2           1.738049        -0.508685        1.331498   
3           1.738049        -0.508685       -0.751034   
4          -0.575358        -0.508685       -0.751034   

   Destination_PSO J318.5-22  Destination_TRAPPIST-1e  VIP_True  
0                  -0.318145                 0.657003 -0.153063  
1                  -0.318145                 0.657003 -0.153063  
2                  -

una vez teniendo los datos escalados, ya se puede hacer una predicción. Iba a escoger adaboost porque salió con mejor accuracy, pero al parecer no soporta el estilo neutron (no maneja NaNs de forma nativa), por lo que usaré random forest.

In [336]:
final_predictions = rf.predict(xTestRealEscalado)
# AI lo de abajo
final_predictions = pd.Series(final_predictions).map({0: False, 1: True})

sin embargo, también tengo que adjuntar los IDs de los participantes. entonces:

In [ ]:
blah = pd.read_csv("test.csv")
# AI para ver como crear un df
participantIDSTestSet = blah["PassengerId"]
final_predictions_for_csv = pd.DataFrame(
    {
        "PassengerId": participantIDSTestSet,
        "Transported": final_predictions
    }
)
final_predictions_for_csv.to_csv("Resultados.csv", index=False)



In [339]:
print(final_predictions_for_csv.shape)

(4277, 2)


Las dimensiones son las esperadas. La precisión del modelo se encuentra en esta misma carpeta.